# What sells a car?

This project aims to determine which factors influence the price of the vehicle the most. The analysis of the project will help the client who posted their advertisements for their vehicle be on the right track when pricing their vehicle for sale. 


## Initialization

In [1]:
# Loading all the libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


Bad key text.latex.preview in file C:\Users\x\anaconda3\envs\praktikum_env\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\x\anaconda3\envs\praktikum_env\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\x\anaconda3\envs\praktikum_env\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is save

In [2]:
# Load the data file into a DataFrame
try:
    df = pd.read_csv('vehicles_us.csv')
except:
    df = pd.read_csv('/datasets/vehicles_us.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/vehicles_us.csv'

In [ ]:
# Dataframe info
df.info()

In [ ]:
# Size of the dataframe
df.shape

### Explore initial data

The dataset contains the following fields:
- `price` — price listed on ads.
- `model_year` — vehicle model year.
- `model` — vehicle model.
- `condition` — conditions of the vehicle.
- `cylinders` — number of cylinders.
- `fuel` — fuel type.
- `odometer` — the vehicle's mileage when the ad was published
- `transmission` — transmission.
- `paint_color` — color of the vehicle.
- `is_4wd` — whether the vehicle has 4-wheel drive (Boolean type).
- `date_posted` — the date the ad was published.
- `days_listed` — from publication to removal.


In [ ]:
# Statistic of the dataframe
df.describe(include='all')

In [ ]:
# print a sample of data
df.sample(5)

In [ ]:
# missing values of the dataframe
df.isna().sum()

In [ ]:
# percentage missing values of the dataframe
df[['model_year','cylinders','odometer','paint_color','is_4wd']].isna().sum()/len(df)

In [ ]:
# initial data correlation 
df.corr()

#### Initialized Data
- total 5 columns missing data.
- with `is_4wd` column missing the most data which is up to 50%.
- `date_posted` should be in datetime datatype.

## Data preprocessing

### date_posted

In [ ]:
# Change date_posted to datetime formate
df['date_posted']= pd.to_datetime(df['date_posted'])

### model_year

In [ ]:
# Create years_posted from date_posted column
df['years_posted']=df['date_posted'].dt.year

In [ ]:
# Fill missing model_year with median of years different from year_posted and model_year.
df['model_year'].fillna(df['years_posted']-((df['years_posted']-df['model_year']).median()),inplace=True)
df['model_year'].isna().sum()

In [ ]:
# Change model_year dtype to int64
df['model_year']=df['model_year'].astype('int64')

In [ ]:
# Drop temporary created column
df = df.drop(['years_posted'], axis=1)

### is_4wd

In [ ]:
# Fill missing values in is_4wd assuming they are all 0 (false)
df['is_4wd'].fillna(0,inplace=True)

In [ ]:
# Change is_4wd dtype to int64
df['is_4wd']=df['is_4wd'].astype('int64')

### cylinder

In [ ]:
# Fill missing values of cylinder groupby type and model median
df['cylinders'] = df.groupby(['type','model'])['cylinders'].apply(lambda x: x.fillna(x.median()))
df['cylinders'].isna().sum()

In [ ]:
# Missing values still exist
df[df['cylinders'].isna()].head()

In [ ]:
# Drop left over missing values
df.dropna(subset=['cylinders'], inplace=True)

In [ ]:
# Change cylinders dtype to int64
df['cylinders']=df['cylinders'].astype('int64')

### odometer

In [ ]:
# Print 0 millage vehicle
df[df['odometer']==0].head()

In [ ]:
# Print missing values odometer
df[df['odometer'].isna()].head()

In [ ]:
# Calculate total missing values of odometer
df['odometer'].isna().sum()

In [ ]:
# Replace 0 odometer vales with NaN
df['odometer'].replace(0,np.nan,inplace=True)

In [ ]:
# Fill odometer missing values group by model_year
df['odometer'] = df.groupby(['model_year'])['odometer'].apply(lambda x: x.fillna(x.median()))
df['odometer'].isna().sum()

In [ ]:
# Drop the 1 left over missing value
df.dropna(subset=['odometer'], inplace=True)

In [ ]:
# Convert Odometer dtype to int64
df['odometer']=df['odometer'].astype('int64')

### paint_color

In [ ]:
# List paint_color values
df['paint_color'].value_counts()

In [ ]:
# Fill missing values with most common color
df['paint_color'].fillna('Others',inplace=True)
# Recheck missing value
df['paint_color'].isna().sum()

### price

In [ ]:
# Print lenght of price listed lower than 100
len(df[df['price'] < 100])

In [ ]:
# Sample of price lower than 100
df.query('price<100').sample(5)

In [ ]:
# Replace price <100 with NaN
for row in df['price']:
    if row < 100:
        df['price'].replace(row,np.nan,inplace=True)

In [ ]:
# Filled those NaN groupby model and type median
df['price'] = df.groupby(['model','type'])['price'].apply(lambda x: x.fillna(x.median()))
df['price'].isna().sum()

In [ ]:
# Drop the 1 left over Null value
df.dropna(subset=['price'], inplace=True)

In [ ]:
# Reset Index
df.reset_index(drop=True, inplace=True)

In [ ]:
# Recheck for dataframe null values
df.isna().sum()

In [ ]:
# Check for dataframe duplicates
df.duplicated().sum()

In [ ]:
# print the general/summary information about the DataFrame
df.info()

**Data preprocessing summarized conclusion**

- Missing values of the model ear are filled with a median of years different from the year posted and model year.
- Missing values of the cylinder are filled group by type and model median.
- Leftover values of the cylinder are dropped because there are only 15 of them.
- Odometer 0 is probably an error. Even brand new cars have some mileage due to transportation from manufacturer to car dealer. Both 0 and missing values are filled group by the model-year median.
- Missing values in paint colour are filled with the most common colour, white.
- Prices lower than 100 are being replaced with groupby model and type median because cars with prices lower than 100 are probably errors during data entry.
- Dataframe has no duplicates at all.
- Model year,is_4wd, odometer, and cylinders datatype changed to Int64.


### Enrich data

In [ ]:
# Create month_posted column from date_posted's months
df['month_posted'] = df['date_posted'].dt.month
# Create days_posted column from date_posted's days
df['days_posted'] = df['date_posted'].dt.dayofweek

In [ ]:
# Add the vehicle's age when the ad was placed
df['car_age'] = ((df['date_posted'] - pd.to_datetime(df.model_year, format='%Y'))/ np.timedelta64(1, 'Y'))+1
# Print car_age statistic
df['car_age'].describe()

In [ ]:
# Add the vehicle's average mileage per year
df['avg_mil']=(df['odometer']/df['car_age'])
# Print average mile statistic
df['avg_mil'].describe().round()

In [ ]:
# List condition values
df['condition'].value_counts()

In [ ]:
# Assign condition id to each conditions
def df_condition_id(condition):
    if condition == 'new':
        return 5
    if condition == 'like new':
        return 4
    if condition == 'excellent':
        return 3
    if condition == 'good':
        return 2
    if condition == 'fair':
        return 1
    if condition == 'salvage':
        return 0

df['condition_id']=df['condition'].apply(df_condition_id).astype('int64')

In [ ]:
# Check condition_id
df['condition_id'].value_counts()

## Exploratory Data Analysis (EDA)
Listed are the main parameters that will be used to study the price factorization.
- Price
- The vehicle's age when the ad was placed
- Mileage
- Number of cylinders
- Condition

In [ ]:
# print a sample of data
df.sample(5)

In [ ]:
# Price statistic
df['price'].describe()

In [ ]:
# Price distribution histogram
sns.histplot(data=df, x="price",bins=50,binrange=(0,50000)).set(title="Price distribution");

In [ ]:
# Vehicles' age distribution histogram
sns.histplot(data=df, x="car_age",bins=20,binrange=(0,40)).set(title="Vehicles' age distribution");

In [ ]:
# Odometer distribution histogram
sns.histplot(data=df, x="odometer",bins=20,binrange=(0,400000)).set(title="Millage distribution");

In [ ]:
# Average mileage distribution histogram
sns.histplot(data=df, x="avg_mil",bins=40,binrange=(0,80000)).set(title="Average mileage distribution");

In [ ]:
# Condition distribution bar plot
sns.countplot(x ='condition', data = df
              ,order = df['condition'].value_counts().index).set(title="Vehicle's condition distribution");

In [ ]:
# Cylinders distribution bar plot
sns.countplot(x ='cylinders', data = df
              ,order = df['cylinders'].value_counts().index).set(title="Vehicle's cylinders distribution");

In [ ]:
# Vehicle type distribution bar plot
sns.countplot(x ='type', data = df
              ,order = df['type'].value_counts().index).set(title="Vehicle's type distribution")
plt.xticks(rotation=90);

In [ ]:
# Vehicle Fuel type distribution bar plot
sns.countplot(x ='fuel', data = df
              ,order = df['fuel'].value_counts().index).set(title="Vehicle's cylinders distribution");

In [ ]:
# Vehicle 4WD distribution bar plot
sns.countplot(x ='is_4wd', data = df
              ,order = df['is_4wd'].value_counts().index).set(title="Vehicle 4WD distribution");

In [ ]:
# Various boxplot distribution on price, vehicle's age, and Days listed.
# plt.figure(figsize=(9,7))
# sns.boxplot(x='paint_color', y='price',data=df).set(title="Color vs Price distribution:boxplot");
# plt.figure(figsize=(9,7))
# sns.boxplot(x='paint_color', y='car_age',data=df).set(title="Vehicles' Age vs Price distribution:boxplot");
# plt.figure(figsize=(9,7))
# sns.boxplot(x='paint_color', y='days_listed',data=df).set(title="Color vs days listed distribution:boxplot");
# plt.figure(figsize=(9,7))
# sns.boxplot(x='type', y='price',data=df).set(title="Type vs price distribution:boxplot");
# plt.xticks(rotation=90);
# plt.figure(figsize=(9,7))
# sns.boxplot(x='condition_id', y='price',data=df).set(title="Condition vs price distribution:boxplot");
# plt.figure(figsize=(9,7))
# sns.boxplot(x='condition_id', y='car_age',data=df).set(title="Condition vs Vehicles' age distribution:boxplot");
# plt.figure(figsize=(9,7))
# sns.boxplot(x='fuel', y='price',data=df).set(title="Fuel type vs price distribution:boxplot")
# plt.figure(figsize=(9,7))
# sns.boxplot(x='is_4wd', y='price',data=df).set(title="4WD vs price distribution:boxplot");

### EDA summarized conclusion
- Boxplot showed a great number of outliers. 
- Outliers need to be eliminated to get normal and better distribution.
- Most vehicle price above 5000 to 12000.
- Most vehicle listed is seven years old.
- Mileage distribution peaked between 100k and 150k miles
- Average mileage is around 10k to 15k.
- Most cars listed are in excellent and good shape.
- Most cars listed have 4 to 8 cylinders.
- Most popular car types are SUVs, trucks, and sedans.
- Majority of the vehicle fuel type is gas.
- The 4WD distribution is pretty much even across the DataFrame.


### Filtered dataframe without outliers

In [ ]:
# Create column list with int64 and float64 datatype
intlist = []
for col in df.columns:
    if df[col].dtype == np.int64 and np.float64:
        intlist.append(col)

In [ ]:
# Getting rid of upper outliers.
filter = np.zeros(len(df), dtype=bool) + True
for outliers in ['car_age','odometer','days_listed','price','avg_mil']:
    Q1 = df[outliers].quantile(0.25)
    Q3 = df[outliers].quantile(0.75)
    IQR = Q3-Q1
    upper_outliers = Q3+1.5*IQR 
    filter[np.where(df[outliers]>upper_outliers)]=False

In [ ]:
# df2 as filtered dataframe without upper outliers
df2 = df[filter]
df2.reset_index(drop=True, inplace=True);

In [ ]:
# Correlation Matrix of dataframe
corr_matrix = df[intlist].corr().abs()

plt.subplots(figsize=(13, 9))
sns.heatmap(corr_matrix,annot=True,fmt='.3g')

# Mask unimportant features
sns.heatmap(corr_matrix, mask=corr_matrix < 1, cbar=False)
plt.show()

In [ ]:
sns.histplot(data=df2,x='car_age',bins=20).set(title='Vehicles age distribution(df2)');

In [ ]:
sns.histplot(data=df2,x='odometer',bins=25).set(title='Odometer distribution(df2)');

In [ ]:
sns.histplot(data=df2,x='price',bins=35).set(title='Price distribution(df2)');

In [ ]:
sns.histplot(data=df2,x='avg_mil',bins=30).set(title='Average mileage distribution(df2)');

In [ ]:
sns.histplot(data=df2,x='model_year',bins=20).set(title='Model year distribution(df2)');

In [ ]:
sns.histplot(data=df2,x='days_listed',bins=25).set(title='Days listed distribution(df2)');

In [ ]:
# New filtered outliers dataframe (df2) Various boxplot distribution on price, vehicle's age, and Days listed.
plt.figure(figsize=(9,7))
sns.boxplot(x='paint_color', y='price',data=df2).set(title="Color vs Price distribution:boxplot");
plt.figure(figsize=(9,7))
sns.boxplot(x='paint_color', y='car_age',data=df2).set(title="Vehicles' Age vs Price distribution:boxplot");
plt.figure(figsize=(9,7))
sns.boxplot(x='paint_color', y='days_listed',data=df2).set(title="Color vs days listed distribution:boxplot");
plt.figure(figsize=(9,7))
sns.boxplot(x='type', y='days_listed',data=df2).set(title="Type vs days listed distribution:boxplot");
plt.figure(figsize=(9,7))
plt.xticks(rotation=90);
sns.boxplot(x='type', y='price',data=df2).set(title="Type vs price distribution:boxplot");
plt.xticks(rotation=90);
plt.figure(figsize=(9,7))
sns.boxplot(x='condition_id', y='price',data=df2).set(title="Condition vs price distribution:boxplot");
plt.figure(figsize=(9,7))
sns.boxplot(x='condition_id', y='car_age',data=df2).set(title="Condition vs Vehicles' age distribution:boxplot");
plt.figure(figsize=(9,7))
sns.boxplot(x='fuel', y='price',data=df2).set(title="Fuel type vs price distribution:boxplot")
plt.figure(figsize=(9,7))
sns.boxplot(x='is_4wd', y='price',data=df2).set(title="4WD vs price distribution:boxplot")
plt.figure(figsize=(9,7))
sns.boxplot(x='is_4wd', y='cylinders',data=df2).set(title="4WD vs cylinders distribution:boxplot");

In [ ]:
# Vehicles age vs price scatterplot
sns.scatterplot(data=df2,x='car_age', y='price', alpha=.01).set(title='Vehicles age vs price');

In [ ]:
# Average millage vs price scatterplot
sns.scatterplot(data=df2,x='avg_mil', y='price', alpha=.01).set(title='Average millage vs price');

In [ ]:
# Odometer vs price scatterplot
sns.scatterplot(data=df2,x='odometer', y='price', alpha=.01).set(title='Odometer vs price');

In [ ]:
# Model_year vs price scatterplot
sns.scatterplot(data=df2,x='model_year', y='price', alpha=.01).set(title='Model_year vs price');

In [ ]:
# Days_listed vs price scatterplot
sns.scatterplot(data=df2,x='price', y='days_listed', alpha=.01).set(title='Model_year vs price');

In [ ]:
# SUV, Sedan, and Truck: Vehicle type vs price scatterplot
plt.figure(figsize=(9, 7))
sns.scatterplot(data=df2[df2['type'].isin(['SUV', 'sedan','truck'])],
                x='car_age', y='price', alpha=.075,hue='type').set(title='Vehicle age of SUV,Sedan,Truck type vs price');

In [ ]:
# SUV, Sedan, and Truck (type):Vehicle's age vs price hexplot
df2.plot(x='car_age', y='days_listed', kind='hexbin', gridsize=20, figsize=(9, 7), sharex=False, grid=True)
plt.title('Vehicle age of SUV,Sedan,Truck type vs Price:hexplot');

In [ ]:
# SUV, Sedan, and Truck (type):Vehicle's age vs price hexplot
df2.plot(x='car_age', y='price', kind='hexbin', gridsize=20, figsize=(9, 7), sharex=False, grid=True)
plt.title('Vehicle age of SUV,Sedan,Truck type vs Price:hexplot');

In [ ]:
# SUV, Sedan, and Truck (type):Vehicle's age vs price hexplot
df2.plot(x='days_listed', y='price', kind='hexbin', gridsize=20, figsize=(9, 7), sharex=False, grid=True)
plt.title('Days_listed of SUV,Sedan,Truck type vs Price:hexplot');

In [ ]:
# Median and mean value of Days_listed on vechicle type pivot table.
df2.pivot_table(index='type', values='days_listed', aggfunc=('mean', 'median')).round(2)

**Summarized New Dataframe without Outliers**

- Correlation matrix showed `model year` and `odometer` has the highest value correlation. Which makes sense because the higher value `odometer` tend to be produced from older vehicles. 4WD and cylinders have high correlation as well, data showed 4WD car has at least more than 6 cylinders. 

- Majority of the vehicle age is approximately 7 years. Green and purple `color` vehicle higher age distribution than majority of the `color`s. Majority of the lower average vehicle age are in better `condition`. Surprisingly, vehicle age of fair `condition` vehicle are higher than vehicle with salvage `condition`. 

- Vehicle `color` orange and yellow showed higher median price compared to the rest of the color. 

- `odometer` distribution looks normal after getting rid of the outliers, peaking at 125k miles. Based on `odometer` vs `price` scatter plot, 75k to 175k miles vehicle are listed around 2500 to 10000 `price` range. 

- Peak distribution of the `price` of the vehicle listed are in range of 5000 to 7500. 

- Majority of the `model year` posted are in year 2011 to 2012. `price` range on later `model year` car tend to be wider and higher compared to car `model year` before 2011. 

- Based on days listed distribution histogram, most of the ads are listed for 30 to 33 days (about 1 month). Distribution seems normal across all the `colors` of the vehicle on day listed. `color` does not have any impact on the days listed. However, the type of the bus has longer day listed on ad vs every other type of the vehicle, median days listed are approximately 50days (about 1 and a half months) based on 'type vs days listed distribution' boxplot. Most of the data listed between 0 to 40 days (about 1 and a half months), in the `price` range of 2500 to 10000. 

- Pickup, Truck, convertible, coupe, and off-road type vehicle showed higher `price` range, approximately 15000. The rest of the `vehicle type` has median `price` lower than 10000. 

- SUV, Sedan, and Truck showed as the most common vehicle listed on the ads. Scatter plot showed that sedan has lowest `price` range among the three types, and truck has the highest `price` range among three `vehicle type`s. 

- Majority SUV, Sedan, and Truck listed are 7 years old, and typical listed days ranged between 25 to 38 days (about 1 month 1 week) with `price` range of 4500 to 9000. 

- `condition` vs `price` distribution boxplot showed new, new, excellent has higher `price` range. 

- Diesel and 'other' `fuel` type showed a higher price` range at 17000 compared to gas, hybrid, and electric `fuel` type vehicle which is the median below 10000. 

### Price factors
Newer or later `model year` vehicle have higher price range. `type` of vehicle are the price factor as well. Vehicle `color` orange and yellow seems to be the price factor as well, purple color lowers the price tag. Vehicle `type` pickup, truck, coupe, offroad, and convertible are the listed at higher price range. Besides that, new, new, excellent `condition` listed at a higher price as well. Lastly, the vehicle `fuel` `type` with Diesel and other `fuel` `type` has a higher price range compared to the rest of the `fuel` `type` vehicle. 

## General conclusion
- Data preprocessing: 

    - Missing values of the `model` year filled with median of years difference from year posted and model year. 

    - Missing values of the `cylinder` filled with group by type and model median. 

    - 15 Left over values of `cylinder` are dropped. 

    - Both 0 and missing values on `odometer` are filled group by model year median. 

    - Missing values in `paint color` are replaced with `'others'` 

    - `Price` lower than 100 are being replaced with group by model and type median. 

    - Dataframe has no duplicates at all. 

    - Model `year, is_4wd, odometer, and cylinders` datatype changed to `Int64`. 

     

     

- My analysis shows these factors have the most impact on vehicle prices: 

    - `color: Yellow or Orange, NOT purple` 

    - `model_year: above 2010` 

    - `vehicle type: pickup, truck, coupe, convertible, or offroad` 

    - `condition: new, like new, or excellent` 

    - `fuel type: diesel or other` 

     

**I recommend we create a quiz or filtered base listing to auto-generated price range for the client who posted their ad on Crankshaft List based on their vehicle's data to list their vehicle within the correct price range, which can help increase their vehicle sales.** 